In [64]:
from langchain_community.agent_toolkits import JsonToolkit, create_json_agent
from langchain_community.tools.json.tool import JsonSpec
from langchain_groq import ChatGroq


In [65]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')

In [66]:
json_spec = JsonSpec(
    dict_={
        "type": "object",
        "properties": {
            "intent": {"type": "string", "description": "The user's action or intent, e.g., CREATE, READ, UPDATE, DELETE."},
            "data": {
                "type": "object",
                "properties": {
                    "category": {"type": "string", "description": "The category of the expense."},
                    "amount": {"type": "number", "description": "The expense amount."},
                    "description": {"type": "string", "description": "Details or notes about the expense."},
                    "date": {"type": "string", "description": "The date of the expense."},
                },
                "required": ["category", "amount", "description"],
            },
        },
        "required": ["intent", "data"],
    }
)


In [67]:
llm = ChatGroq(model='llama-3.3-70b-versatile')
toolkit = JsonToolkit(spec=json_spec)


In [68]:
json_agent_executor = create_json_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
)

In [69]:
# llm1_output = 'UserId: 987h78\nAction: CREATE\nAmount: 50.00\nCategory: Leisure\nDate: 13-01-2025\nNotes: football expense of 50 from yesterday (13-01-2025)'

# response = json_agent_executor.invoke(f"Convert the following into a JSON object: {llm1_output}")


In [70]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import SystemMessage, HumanMessage
from langchain_core.output_parsers.json import JsonOutputParser
from pydantic import BaseModel

In [71]:
class ExpenseSchema(BaseModel):
    UserId: str
    Action: str
    Amount: float
    Category: str
    Date: str
    Notes: str

In [72]:
output_parser = JsonOutputParser(pydantic_object=ExpenseSchema)

In [73]:
text_input = """UserId: 987h78
Action: CREATE
Amount: 50.00
Category: Leisure
Date: 13-01-2025
Notes: football expense of 50 from yesterday (13-01-2025)"""

prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You are a helpful assistant. Convert structured text into valid JSON."),
        HumanMessage(content="Input text: {input}\nOutput in JSON format following the defined schema."),
    ]
)

In [74]:
messages = prompt_template.format_messages(input=text_input)

In [75]:
response = llm(messages)

In [76]:
# json_result = output_parser.parse(response.content)

# print(json_result)

In [88]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers.json import JsonOutputParser
from pydantic import BaseModel, ValidationError

# Define the schema with Pydantic
class ExpenseSchema(BaseModel):
    userId: str
    action: str
    amount: float
    category: str
    date: str
    description: str

# Create the output parser
output_parser = JsonOutputParser(pydantic_object=ExpenseSchema)



# Input text
text_input = """UserId: 987h78
Action: DELETE
Amount: Not specified
Category: Miscellaneous
Date: 14-01-2025
Notes: Delete user account on today (14-01-2025)"""

# Build a structured prompt template
prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(
            """You are a helpful assistant. Your task is to convert structured text into a JSON object that conforms to a specific schema.
                If numeric values are not provided, they should be set to 0.0.
                If a date is not provided, it should be set to the current date.
                If a category is not provided, it should be set to 'Miscellaneous'.
                If a description is not provided, it should be set to an empty string.
            """

        ),
        HumanMessagePromptTemplate.from_template(
            "Structured text input:\n{text}\n\n"
            "Please generate a JSON object that matches the following schema:\n"
            "{schema}"
            "Output only valid JSON."
        )
    ]
)

# Generate the formatted prompt messages
messages = prompt_template.format_messages(text=text_input, schema=ExpenseSchema.model_json_schema())

# Generate the LLM response
response = llm(messages)

# Validate the response using JsonOutputParser
try:
    json_result = output_parser.parse(response.content)
    print(json_result)
except ValidationError as e:
    print("ValidationError:", e)
except Exception as e:
    print("Other Error:", e)


{'userId': '987h78', 'action': 'DELETE', 'amount': 0.0, 'category': 'Miscellaneous', 'date': '14-01-2025', 'description': 'Delete user account on today (14-01-2025)'}
